#Trabajo Practico 2

Ejercicios a realizar: 3 11 17 39 29 5 \\


##Instalaciones

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Ign:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubun

##Imports

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from google.colab import drive 
from pyspark.sql import SQLContext
from nltk import word_tokenize
from nltk.util import ngrams
from pyspark.ml.feature import NGram
import pandas as pd
import nltk

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


##Descargas

In [ ]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

###Contents

In [ ]:
contents = spark.read.format("csv").option("header", True).load('gdrive/My Drive/Wikipedia dataset 2c2021.zip (Unzipped Files)/contents.csv')
rdd_contents = contents.rdd

###Contents text sample

In [ ]:
contents_text_sample = spark.read.format("csv").option("header", True).load('gdrive/My Drive/Wikipedia dataset 2c2021.zip (Unzipped Files)/contents_text_sample.csv')
rdd_contents_text_sample = contents_text_sample.rdd

###Geo tags

In [ ]:
geo_tags = spark.read.format("csv").option("header", True).load('gdrive/My Drive/Wikipedia dataset 2c2021.zip (Unzipped Files)/geo_tags.csv')
rdd_geo_tags = geo_tags.rdd

###Logs

In [ ]:
logs = spark.read.format("csv").option("header", True).load('gdrive/My Drive/Wikipedia dataset 2c2021.zip (Unzipped Files)/logs.csv')
rdd_logs = logs.rdd

###Page Links

In [ ]:
id='1Nxk1FhbEpwAqM9070I-0ESVa4wKEVMaB'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('contents.csv')
contents_pd = pd.read_csv('contents.csv')

In [ ]:
id='1yCOlwU0KD-VziLLNzUu1FZiR1T3QgJNZ'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('pagelinks_sample.csv')
pagelinks = pd.read_csv('pagelinks_sample.csv')

## 3) Realizar un análisis de stopwords del contenido de texto de la Wikipedia. En este punto esperamos que analicen, dada la frecuencia de los términos que hay en la wikipedia cuales deberian ser considerados stop words. 

Para resolver este ejercicio filtre los textos y los titulos que no eran vacios y luego los uni y obtuve la probabilidad de aparicion de cada palabra con casos favorables/casos totales. \
Para determinar cuales palabras eran stopword fui probando con distintas probabilidades de aparicion y filtrando y me quede con un filtro que aproximadamente se adapta a las stopwords mas comunes.

In [ ]:
textos = rdd_contents_text_sample.filter(lambda x: x.text != None).map(lambda x: x.text)
titulos = rdd_contents_text_sample.filter(lambda x: x.title != None).map(lambda x: x.title)
union = textos.union(titulos)


In [ ]:
palabras = union.flatMap(lambda x: x.split())
total_palabras = palabras.count()


In [ ]:
frecuencia_palabras = palabras.map(lambda x: (x.lower(), 1)).reduceByKey(lambda x,y: x+y)

In [ ]:
stopwords = frecuencia_palabras.filter(lambda x: (((x[1])/total_palabras)*100) > 0.09)
stopwords.takeOrdered(10, lambda x: x[1])

[('ser', 17462),
 ('durante', 17531),
 ('parte', 17787),
 ('son', 17801),
 ('años', 18003),
 ('ha', 18563),
 ('le', 18678),
 ('esta', 18815),
 ('and', 19129),
 ('hasta', 19197)]

## 11) Generar una función genérica que dado un n nos permita obtener un RDD con los n-gramas del contenido de texto de wikipedia y su frecuencia

In [ ]:
textos = rdd_contents_text_sample.filter(lambda x: x.text != None).map(lambda x: x.text)
titulos = rdd_contents_text_sample.filter(lambda x: x.title != None).map(lambda x: x.title)
union = textos.union(titulos)

Con word tokenize me tarda 10 min los 2gram \
Con split y el espacio tarda 1min30 asi que use split


In [ ]:
def ngramas(cantidad):
  palabras_parseadas = union.flatMap(lambda x: ngrams(x.split(), cantidad))
  return palabras_parseadas.map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)


In [ ]:
pip install nltk==3.4

In [ ]:
ngramas(3).take(5)

[(('D.', 'Anderson]]}}', 'encontró'), 1),
 (('de', 'niebla]]', '—un'), 1),
 (('partículas]]', 'donde', 'los'), 1),
 (('[[positrón|positrones]]', 'que', 'se'), 1),
 (('que', 'se', 'mueven'), 26)]

##17) . Para aquel contenido georeferenciado publicado anónimamente indicar por país, cuántas IPs de usuarios corresponden a IPv4 y cuantas a IPv6.

Los contenidos georeferenciados son los geo tags que estan en contents, entonces uni usando el id para tener el pais de cada ID y su tipo de IP

In [ ]:
geo_tags_id_pais = rdd_geo_tags.filter(lambda x: x.gt_country != None).map(lambda x: (x.gt_page_id, x.gt_country))

Aca filtro los id que son vacios porque se supone que si esta por IP no deberia tener un ID, hay algunos registros que si tienen ID e IP pero me fije y son unos 50 que se leen mal, tienen cualquier cosa en el ID y en el IP

In [ ]:
contents_filtrado = rdd_contents.filter(lambda x: x.revisor_ip != None and x.revisor_id == None).map(lambda x: (x.id, x.revisor_ip))
contents_id_ip = contents_filtrado.map(lambda x: (x[0], "IPV6" if ":" in x[1] else "IPV4"))

In [ ]:
rdd_id_ip_pais = contents_id_ip.join(geo_tags_id_pais).map(lambda x: (x[1], 1))

In [ ]:
frecuencia_ip = rdd_id_ip_pais.reduceByKey(lambda x,y: x+y)
final = frecuencia_ip.map(lambda x: (x[0][1], (x[0][0], x[1]))).reduceByKey(lambda x,y: (x,y))

In [ ]:
final.take(5)

[('AF', ('IPV4', 1)),
 ('CZ', ('IPV4', 1)),
 ('SB', ('IPV4', 1)),
 ('ES', (('IPV6', 4), ('IPV4', 557))),
 ('SN', ('IPV4', 1))]

## 29) Para los contenidos visibles en wikipedia, cuales son los artículos que tienen la máxima y mínima distancia entre ids de su revisión actual y la anterior

Aca lo mismo, filtro que haya id de la revision anterior, de la actual y que estas y el id sea numerico, por estos registros que se leen y tienen cualquier cosa

In [ ]:
contents_con_ids = rdd_contents.filter(lambda x: x.revision_id != None and x.parent_revision != None and x.revision_id.isnumeric() and x.parent_revision.isnumeric() \
                                       and x.id.isnumeric()).cache()

In [ ]:
id_con_distancias = contents_con_ids.map(lambda x: (x.id, int(x.revision_id) - int(x.parent_revision) if int(x.revision_id) > int(x.parent_revision) \
                                                    else int(x.parent_revision) - int(x.revision_id))).cache()
id_con_distancias.take(3)

[('5', 16676), ('7', 7582), ('10', 141661)]

In [ ]:
distancia_minima = (id_con_distancias.takeOrdered(1, key=lambda x: x[1]))[0][1]

In [ ]:
registros_distancia_minima = id_con_distancias.filter(lambda x: x[1] == distancia_minima)  

Devuelvo un rdd con las distancias minimas, su ID y su titulo

In [ ]:
rdd_distancias_minimas = contents_con_ids.map(lambda x: (x.id, x.title)).join(registros_distancia_minima).map(lambda x: ( x[0],  x[1][0], x[1][1])) 
rdd_distancias_minimas.take(3)

[('299', 'Albaida (Valencia)', 1),
 ('336', 'Bomba lógica', 1),
 ('545', 'Cono', 1)]

In [ ]:
distancia_maxima = id_con_distancias.reduce(lambda x,y: x if x[1]>y[1] else y)[1]  

In [ ]:
registros_distancia_maxima = id_con_distancias.filter(lambda x: x[1] == distancia_maxima)   

In [ ]:
rdd_distancias_maximas = contents_con_ids.map(lambda x: (x.id, x.title)).join(registros_distancia_maxima).map(lambda x: ( x[0],  x[1][0], x[1][1]))  
rdd_distancias_maximas.take(1)

[('30623', 'CEDA', 137146746)]

##39) . ¿Cuál es la acción más realizada por usuarios que no están registrados?

In [ ]:
acciones_usuarios_no_registrados = rdd_logs.filter(lambda x:  x.contributor_id==None and x.contributor_ip != None and x.action!=None)


In [ ]:
frecuencia_acciones = acciones_usuarios_no_registrados.map(lambda x: (x.action,1)).reduceByKey(lambda x,y: x+y)

In [ ]:
frecuencia_acciones.reduce(lambda x,y: x if x[1]>y[1] else y)

('create', 123293)

## 5) Considerando el pagelink_sample.csv representar como un grafo en Spark los contenidos de wikipedia (considerando los contenidos como nodos y los links como aristas) como una lista de adyacencia y mostrar un recorrido en la estructura.


In [ ]:
pagelinks_booleano_reducido = pagelinks['pl_title'].isin(contents_pd['title'])
df_pagelinks_reducido = pagelinks[pagelinks_booleano_reducido]


In [ ]:
parquet_pagelinks = df_pagelinks_reducido.to_parquet('pagelinks_reducido.parquet')

In [ ]:
pagelinks_reducido = spark.read.format("parquet").option("header", True).load('pagelinks_reducido.parquet')

In [ ]:
rdd_pagelinks_reducido = pagelinks_reducido.rdd

In [ ]:
del df_pagelinks_reducido
del pagelinks
del contents_pd

In [ ]:
tupla_pagelinks = rdd_pagelinks_reducido.map(lambda x:(x.pl_title, x.pl_from))
tupla_contents = rdd_contents.map(lambda x: (x.title, x.id))

In [ ]:
ids_destino = tupla_pagelinks.join(tupla_contents)

In [ ]:
grafo = ids_destino.map(lambda x: (x[1][0], x[1][1])).groupByKey().cache()

In [ ]:
def camino_random(id, cantidad):
  i = 0
  rdd_inicial = grafo.filter(lambda x: x[0] == id)
  rdd_final = rdd_inicial.map(lambda x: x[0])
  vecinos = rdd_inicial.flatMap(lambda x: list(x[1]))
  vecino_aleatorio = vecinos.takeSample(False, 1)
  while vecino_aleatorio and i<cantidad:
    rdd_vecino = grafo.filter(lambda x: x[0] == int(vecino_aleatorio[0]))
    rdd_final = rdd_final.union(rdd_vecino.map(lambda x: x[0]))
    vecinos = rdd_vecino.flatMap(lambda x: list(x[1]))
    vecino_aleatorio = vecinos.takeSample(False, 1)
    i+= 1
  return rdd_final.collect()
    

In [ ]:
camino_random(7965656, 5)

[7965656, 21612, 136980, 50154, 2467193, 9967387]

In [ ]:
grafo.map(lambda x: (x[0],list(x[1]))).collect()

[(9721272, ['246593', '5291797', '184934']),
 (9206216,
  ['6555398',
   '1034805',
   '57328',
   '2458720',
   '833049',
   '760205',
   '8746763',
   '4149312',
   '2307947',
   '97499',
   '1968216',
   '4964556',
   '278230',
   '9814772',
   '157421',
   '500991',
   '644949',
   '1584357',
   '56986',
   '411602',
   '6991121',
   '5230981',
   '2521295',
   '89618',
   '964306',
   '4254217',
   '426405',
   '2028200',
   '106726',
   '176825',
   '237468',
   '5858606',
   '792610',
   '142688',
   '2225541',
   '6217156',
   '77015',
   '1819571',
   '2076231',
   '5291797',
   '8690676',
   '1586861',
   '256961',
   '2642540',
   '638456',
   '83175',
   '3356935',
   '4573057',
   '283080',
   '315100',
   '5071905',
   '27550',
   '8655499',
   '8542086',
   '72658',
   '1063209',
   '377893',
   '480308',
   '1751126',
   '367028',
   '3506038',
   '79565',
   '2190548',
   '163631',
   '2022849',
   '8016428',
   '443182',
   '181450',
   '400138',
   '201245',
   '5374